In [1]:
# ▶ 1. 필수 라이브러리
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score
import warnings
warnings.filterwarnings('ignore')

In [3]:
df1 = pd.read_csv('병합/merged_final_train_셀_추가병합.csv')

In [7]:
df1

,기준년월,ID,남녀구분코드,회원여부_이용가능,회원여부_이용가능_CA,회원여부_이용가능_카드론,소지여부_신용,소지카드수_유효_신용,소지카드수_이용가능_신용,입회일자_신용,...,금액_할부전환_R6M,금액_할부전환_R3M,신청건수_ATM_CA_B0,신청건수_ATM_CL_B0,승인거절건수_B0M,승인거절건수_한도초과_B0M,승인거절건수_BL_B0M,승인거절건수_BL_R3M,승인거절건수_입력오류_R3M,Segment.1
0,201807,TRAIN_000000,2,1,1,0,1,1,1,20130101,...,0,0,0,0,0,0,0,0,0,D
1,201807,TRAIN_000001,1,1,1,1,1,1,1,20170801,...,0,0,0,0,0,0,0,0,0,E
2,201807,TRAIN_000002,1,1,1,0,1,1,1,20080401,...,0,0,0,0,0,0,0,0,0,C
3,201807,TRAIN_000003,2,1,1,0,1,2,2,20160501,...,0,0,0,0,0,0,0,0,0,D
4,201807,TRAIN_000004,2,1,1,1,1,1,1,20180601,...,0,0,0,0,0,0,0,0,0,E
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2399995,201812,TRAIN_399995,2,1,1,1,1,1,1,20010701,...,0,0,0,0,0,0,0,0,0,E
2399996,201812,TRAIN_399996,2,1,1,1,1,1,1,20170701,...,0,0,0,0,0,0,0,0,0,D
2399997,201812,TRAIN_399997,1,1,1,0,1,1,1,20090501,...,0,0,0,0,0,0,0,0,0,C
2399998,201812,TRAIN_399998,1,1,1,1,1,1,1,20130101,...,0,0,0,0,0,0,0,0,0,E


In [10]:
df2 = df1[[
    'ID',
    '_2순위카드이용금액',
    '이용금액_오프라인_B0M',
    '정상입금원금_B5M',
    '이용금액대',
    '이용금액_오프라인_R3M',
    '이용금액_오프라인_R6M',
    '정상입금원금_B2M',
    '_3순위업종_이용금액',
    '_2순위업종_이용금액',
    '_2순위쇼핑업종_이용금액',
    '최대이용금액_일시불_R12M',
    '_1순위업종_이용금액',
    '_3순위쇼핑업종_이용금액',
    '쇼핑_도소매_이용금액',
    '이용건수_오프라인_R6M',
    '_1순위교통업종_이용금액',
    '연체입금원금_B0M',
    '청구금액_R6M',
    '청구금액_B0',
    '평잔_일시불_6M',
    '월중평잔_일시불',
    '잔액_일시불_B0M',
    '입회일자_신용',
    '최종카드발급경과월',
    '_1순위카드이용건수',
    '월상환론한도금액',
    'CA한도금액',
    '카드이용한도금액_B2M',
    '연체입금원금_B5M',
    '연속유실적개월수_기본_24M_카드',
    '이용금액_체크_R12M',
    '할부금액_무이자_3M_R12M',
    '최대이용금액_체크_R12M',
    '연체입금원금_B2M',
    '탈회횟수_발급1년이내',
    '_2순위신용체크구분',
    '이용금액_선결제_B0M',
    '이용금액_할부_무이자_R12M',
    '이용개월수_할부_무이자_R6M',
    '선입금원금_B5M',
    '최종이용일자_CA_년',
    '이용금액_CA_R3M',
    '할인금액_청구서_B0M',
    'Segment'
]]

In [11]:
df2

,ID,_2순위카드이용금액,이용금액_오프라인_B0M,정상입금원금_B5M,이용금액대,이용금액_오프라인_R3M,이용금액_오프라인_R6M,정상입금원금_B2M,_3순위업종_이용금액,_2순위업종_이용금액,...,탈회횟수_발급1년이내,_2순위신용체크구분,이용금액_선결제_B0M,이용금액_할부_무이자_R12M,이용개월수_할부_무이자_R6M,선입금원금_B5M,최종이용일자_CA_년,이용금액_CA_R3M,할인금액_청구서_B0M,Segment
0,TRAIN_000000,0,4043,9205,120,4131,11097,16125,672,1408,...,0,체크,0,5828,2,0,2018,37417,0,D
1,TRAIN_000001,0,3980,2546,40,14339,18638,2420,1659,2083,...,0,기타,0,2076,0,0,2017,0,163,E
2,TRAIN_000002,0,4524,16949,120,12250,29192,14448,1362,1539,...,0,체크,0,10561,2,0,2018,43002,0,C
3,TRAIN_000003,0,3975,8418,120,10408,18056,13043,774,2284,...,0,체크,0,12098,3,0,2018,47830,0,D
4,TRAIN_000004,0,0,0,0,0,787,0,0,0,...,0,체크,0,0,0,0,0,0,0,E
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2399995,TRAIN_399995,0,0,0,0,0,0,0,0,0,...,0,체크,0,0,0,0,0,0,0,E
2399996,TRAIN_399996,0,4676,21831,120,25205,60373,10764,1547,1810,...,0,기타,0,1888,0,0,0,0,0,D
2399997,TRAIN_399997,0,4516,3269,75,13267,32036,6106,2302,2315,...,0,기타,0,27617,2,0,0,0,0,C
2399998,TRAIN_399998,0,0,0,0,0,0,0,0,0,...,0,기타,0,0,0,0,0,0,0,E


In [12]:
# 저장
df2.to_csv('병합/New_unique_ID_Seg.csv', index=False, encoding='utf-8-sig')

In [13]:
df3 = pd.read_csv('병합/New_unique_ID_Seg.csv')

In [14]:
df3

,ID,_2순위카드이용금액,이용금액_오프라인_B0M,정상입금원금_B5M,이용금액대,이용금액_오프라인_R3M,이용금액_오프라인_R6M,정상입금원금_B2M,_3순위업종_이용금액,_2순위업종_이용금액,...,탈회횟수_발급1년이내,_2순위신용체크구분,이용금액_선결제_B0M,이용금액_할부_무이자_R12M,이용개월수_할부_무이자_R6M,선입금원금_B5M,최종이용일자_CA_년,이용금액_CA_R3M,할인금액_청구서_B0M,Segment
0,TRAIN_000000,0,4043,9205,120,4131,11097,16125,672,1408,...,0,체크,0,5828,2,0,2018,37417,0,D
1,TRAIN_000001,0,3980,2546,40,14339,18638,2420,1659,2083,...,0,기타,0,2076,0,0,2017,0,163,E
2,TRAIN_000002,0,4524,16949,120,12250,29192,14448,1362,1539,...,0,체크,0,10561,2,0,2018,43002,0,C
3,TRAIN_000003,0,3975,8418,120,10408,18056,13043,774,2284,...,0,체크,0,12098,3,0,2018,47830,0,D
4,TRAIN_000004,0,0,0,0,0,787,0,0,0,...,0,체크,0,0,0,0,0,0,0,E
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2399995,TRAIN_399995,0,0,0,0,0,0,0,0,0,...,0,체크,0,0,0,0,0,0,0,E
2399996,TRAIN_399996,0,4676,21831,120,25205,60373,10764,1547,1810,...,0,기타,0,1888,0,0,0,0,0,D
2399997,TRAIN_399997,0,4516,3269,75,13267,32036,6106,2302,2315,...,0,기타,0,27617,2,0,0,0,0,C
2399998,TRAIN_399998,0,0,0,0,0,0,0,0,0,...,0,기타,0,0,0,0,0,0,0,E
